In [1]:
import os
import pandas as pd
from PIL import Image
import sys
import requests
from zipfile import ZipFile
from io import BytesIO

In [2]:
#install and import wandb for data collecting
!pip install wandb
import wandb

wandb.login()


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


wandb: Currently logged in as: luca-porta (porta-luca). Use `wandb login --relogin` to force relogin


True

In [3]:
#download and extract project repository

url= "https://github.com/marcomag416/MLDL/archive/refs/heads/main.zip"

# Send a GET request to the URL
response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './MLDL-main')

Download and extraction complete!


In [4]:
cityscape_dataset_path = "./dataset/Cityscapes/Cityspaces"
# Define paths
gta_dataset_path = "./dataset/"
images_path = gta_dataset_path + 'GTA5/images'
labels_path = gta_dataset_path + 'GTA5/labels'

In [5]:
# Initialize lists to hold data
data = []

# Load images and corresponding masks
for image_filename in os.listdir(images_path):
    if image_filename.endswith('.png'):
        image_path = os.path.join(images_path, image_filename)
        mask_path = os.path.join(labels_path, image_filename)

        # Check if corresponding mask file exists
        if os.path.exists(mask_path):
            # Open image and mask to ensure they can be loaded (optional, for validation)
            try:
                image = Image.open(image_path)
                mask = Image.open(mask_path)

                # Add data to list
                data.append({
                    'image_path': image_path,
                    'mask_path': mask_path
                })
            except Exception as e:
                print(f"Error loading {image_path} or {mask_path}: {e}")

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('./gta5_segmentation_dataset.csv', index=False)

print("Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'")

Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'


In [6]:
#import from packages
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from torch.optim.lr_scheduler import PolynomialLR
from timeit import default_timer as timer
from torch.autograd import Variable

#other imports
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017
from models.bisenet.build_bisenet import BiSeNet
from models.domainAdaptationModule import DomainAdaptationModule

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the custom dataset class
class GTASegmentationDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = self._create_label_mapping()

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data_frame.iloc[idx, 0]
        mask_name = self.data_frame.iloc[idx, 1]

        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('RGB')

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented['image'], augmented['mask']

        mask = self._map_mask(np.array(mask))

        if self.transform:
            # Convert mask to tensor without normalization
            mask = torch.from_numpy(mask).permute(2, 0, 1).float()
            mask = mask[0]

        return image, mask

    def _create_label_mapping(self):
        label_mapping = {label.color: label.ID for label in GTA5Labels_TaskCV2017.list_}
        label_mapping[(0, 0, 0)] = 255  # Ensure unmapped colors go to 'unlabeled'
        return label_mapping

    def _map_mask(self, mask):
        new_mask = np.zeros_like(mask)
        for color, label_id in self.label_mapping.items():
            color_mask = np.all(mask == color, axis=-1)
            new_mask[color_mask] = label_id  # Use label_id instead of color
        return new_mask



# Define paths
csv_file = './gta5_segmentation_dataset.csv'

# Define image transformations
transform = A.Compose([
    A.Resize(720, 1280),
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.RandomResizedCrop(height=720, width=1280, scale=(0.8, 1.0)),
        A.NoOp()
    ], p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Create the dataset and dataloader
train_dataset = GTASegmentationDataset(csv_file=csv_file, transform=transform)

In [8]:
class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)

        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']

        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_dataset = Cityscapes(root_dir=cityscape_dataset_path, split='val', transforms=image_transforms)


print(f"Val_Dataset size: {len(val_dataset)}")

Val_Dataset size: 500


In [9]:
def train(model, optimizer_train, dataloader, loss_fn_train):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    total = 0

    for idx, (inputs_train, targets_train) in enumerate(dataloader):
        inputs_train = inputs_train.to(device)
        targets_train = targets_train.to(device, dtype=torch.long)  # Move data to the appropriate device

        optimizer_train.zero_grad()  # Zero out gradients from the previous iteration
        outputs_train, _, _ = model(inputs_train)  # Forward pass
        # print( "train")
        loss = loss_fn_train(outputs_train, targets_train)  # Calculate the loss

        loss.backward()  # Backward pass
        optimizer_train.step()  # Update the weights

        wandb.log({"train/Batch loss": loss})

        train_loss += loss.item() * inputs_train.size(0)  # Accumulate the total loss
        _, predicted_train = outputs_train.max(1)
        total += targets_train.size(0)

    # Calculate average loss for the epoch
    avg_loss = train_loss / total

    wandb.log({"train/Epoch loss": avg_loss})

    return avg_loss


def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)

    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)

    # Calculate average loss
    avg_loss = test_loss / total

    wandb.log({})

    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes

    wandb.log({"val/Validation loss": avg_loss, "val/mIoU": miou})

    return avg_loss, miou, mean_iou

In [10]:
from build_bisenet import BiSeNet
from torch import nn
from torch.optim.lr_scheduler import PolynomialLR


# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

#save hyperparameters
config = {
    "learning_rate": 1e-3,
    "max_epochs": 50,
    "batch_size": 8,
    "weight_decay": "None",
    "dataset": "Cityscapes",
    "scheduler": "None",
    "optimizer": "Adam",
    "transformations": "gaussian blur, horizontal flip"
}


# create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
#scheduler = PolynomialLR(optimizer, total_iters=config["max_epochs"], power=config["polyPower"])
scheduler = None

In [11]:
from timeit import default_timer as timer

#run names
project_name = "bisenet_gta_horizFlip_rc_jitter"
run_name = "horizFlip_rc_jitter"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

top_miou = 0

"""
# Setup training and save the results
for epoch in range(config["max_epochs"]):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if(top_miou < miou):
      torch.save(model.state_dict(), f"./drive/MyDrive/Colab Notebooks/model_weights/{project_name}/{run_name}_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%")
    for i, value in enumerate(mean_iou):
        print(f"Class: {i}, mIoU: {value:.4f}")
"""

# Define the subfolder name
subfolder_name = "model_states_horizFlip_rc_jitter"

# Create the subfolder if it doesn't exist
os.makedirs(subfolder_name, exist_ok=True)

for epoch in range(config["max_epochs"]):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
        scheduler.step()
    # Save model state every 5 epochs
    if top_miou < miou:
        top_miou = miou
        model_path = os.path.join(subfolder_name, f"{project_name}_{run_name}_epoch{epoch}.pth")
        torch.save(model.state_dict(), model_path)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%")
    for i, value in enumerate(mean_iou):
        print(f"Class: {i}, mIoU: {value:.4f}")


# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

Epoch: 1, Loss: 1.3908826417922973, mIoU: 13.04%
Class: 0, mIoU: 0.4380
Class: 1, mIoU: 0.0106
Class: 2, mIoU: 0.4449
Class: 3, mIoU: 0.0233
Class: 4, mIoU: 0.0002
Class: 5, mIoU: 0.0223
Class: 6, mIoU: 0.0010
Class: 7, mIoU: 0.0000
Class: 8, mIoU: 0.6624
Class: 9, mIoU: 0.0472
Class: 10, mIoU: 0.5583
Class: 11, mIoU: 0.0000
Class: 12, mIoU: 0.0000
Class: 13, mIoU: 0.2414
Class: 14, mIoU: 0.0276
Class: 15, mIoU: 0.0000
Class: 16, mIoU: 0.0000
Class: 17, mIoU: 0.0000
Class: 18, mIoU: 0.0000
Epoch: 2, Loss: 1.2604452934265136, mIoU: 16.97%
Class: 0, mIoU: 0.4698
Class: 1, mIoU: 0.2931
Class: 2, mIoU: 0.6200
Class: 3, mIoU: 0.0275
Class: 4, mIoU: 0.0062
Class: 5, mIoU: 0.0914
Class: 6, mIoU: 0.0279
Class: 7, mIoU: 0.0008
Class: 8, mIoU: 0.6390
Class: 9, mIoU: 0.0806
Class: 10, mIoU: 0.6630
Class: 11, mIoU: 0.0334
Class: 12, mIoU: 0.0000
Class: 13, mIoU: 0.2373
Class: 14, mIoU: 0.0334
Class: 15, mIoU: 0.0000
Class: 16, mIoU: 0.0000
Class: 17, mIoU: 0.0000
Class: 18, mIoU: 0.0000
Epoch: 3, 

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/Batch loss,█▆▆▅▄▃▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂▂▁▁▁▁▁▆▂▁
train/Epoch loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁
val/Validation loss,▄▃▁▂▅▃▃▄▁▂▁▁▂▂▂▁▂▂▇▆▄▂▆▄▄▁▂▁▁▂█▂▃▂▁▂▂▇▂▂
val/mIoU,▁▄▅▅▄▄▅▅▆▆▆▇▇▆▅▆▇▆▄▅▆▆▄▅▄▇▇▇█▇▂▆▆▇█▇▇▁▆█
Epoch,50
train/Batch loss,0.11292
train/Epoch loss,0.10308
val/Validation loss,0.92821
val/mIoU,0.23246
